In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi14QWLtzQC2nF0B2gQTojxukve0byW1qgx0 

# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Automatic Chain-of-Thought (Auto-CoT) Prompting

Automatic Chain-of-Thought (Auto-CoT) Prompting, introduced by Zhang et al. (2022), is an innovative method that enhances the reasoning capabilities of large language models (LLMs) by automatically generating effective reasoning paths.

## How Does Auto-CoT Work?

Imagine you're teaching a student to solve complex problems. Instead of manually creating examples of step-by-step reasoning for every possible problem type, you could be more efficient by having the student generate their own reasoning examples first, then use those as templates.

Auto-CoT follows this logic in three key steps:

1. **Zero-Shot Generation**: The LLM is first prompted to solve a problem using zero-shot chain-of-thought reasoning (thinking step by step without examples).

2. **Example Construction**: These zero-shot reasoning paths are collected and used to automatically construct a few-shot prompt with multiple exemplars that include both questions and their reasoning chains.

3. **Improved Reasoning**: When faced with a new test question, the model uses the automatically constructed few-shot examples as guidance, leading to more structured and effective reasoning.

This approach eliminates the need for human-crafted reasoning examples while still harnessing the performance benefits of few-shot chain-of-thought prompting, making it both efficient and scalable for complex reasoning tasks.

In [4]:
auto_cot_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate solutions using Automatic Chain-of-Thought (Auto-CoT) Prompting for a given user question.

    Auto-CoT uses a Zero-Shot prompt to automatically generate chains of thought. These generated CoTs are then used to build a Few-Shot CoT prompt for the test sample.

    Please structure your response in the following format:

    Step 1: Generate Zero-Shot CoT
    [Use a Zero-Shot prompt to generate a chain of thought for the given question.]

    Step 2: Construct Few-Shot CoT Prompt
    [Use the generated Zero-Shot CoT to build a Few-Shot CoT prompt with examples.]

    Step 3: Solve with Few-Shot CoT
    [Solve the original question using the constructed Few-Shot CoT prompt.]

    Step 4: Final Solution
    [Provide the final solution with a brief explanation.]

    Example:
    Question: "A baker has 12 eggs. If he uses 3 eggs for each cake, how many cakes can he make?"

    Step 1: Generate Zero-Shot CoT
    Let's think step by step. If we divide the total eggs by the eggs per cake, we get the number of cakes. 12 eggs / 3 eggs/cake = 4 cakes.

    Step 2: Construct Few-Shot CoT Prompt
    Example 1:
    Q: A student has 15 pencils. If they give 5 pencils to each friend, how many friends get pencils?
    A: Let's think step by step. 15 pencils / 5 pencils/friend = 3 friends.
    Example 2:
    Q: A farmer has 20 apples. If they put 4 apples in each basket, how many baskets are needed?
    A: Let's think step by step. 20 apples / 4 apples/basket = 5 baskets.
    Q: A baker has 12 eggs. If he uses 3 eggs for each cake, how many cakes can he make?
    A:

    Step 3: Solve with Few-Shot CoT
    Let's think step by step. 12 eggs / 3 eggs/cake = 4 cakes.

    Step 4: Final Solution
    The baker can make 4 cakes.
    """),
    ("human", "{question}"),
    ("assistant", """
    I will use Automatic Chain-of-Thought (Auto-CoT) Prompting to solve this problem.

    Step 1: Generate Zero-Shot CoT
    [Generate a Zero-Shot CoT for the question.]

    Step 2: Construct Few-Shot CoT Prompt
    [Build a Few-Shot CoT prompt using the generated Zero-Shot CoT.]

    Step 3: Solve with Few-Shot CoT
    [Solve the question with the constructed Few-Shot CoT.]

    Step 4: Final Solution
    [Provide the final solution.]
    """)
])


# Suitable Questions for Automatic Chain-of-Thought (Auto-CoT)

For Automatic Chain-of-Thought (Auto-CoT), the most suitable questions are those that benefit from explicit step-by-step reasoning that can be automatically generated and then used as examples. Here are types of questions that would be particularly well-suited to this approach:

### Mathematical Calculations
- Arithmetic problems requiring multiple operations
- Word problems that can be broken down into discrete steps
- Percentage calculations with multiple variables
- Rate, time, and distance problems with clear solution paths

### Logical Reasoning
- Multi-step deduction problems
- Syllogisms requiring careful tracking of premises
- Sequence completion with identifiable patterns
- Conditional reasoning problems (if-then scenarios)

### Procedural Analysis
- Problems involving defined algorithms or procedures
- Step-counting problems with clear rules
- Problems requiring systematic elimination of options
- Tasks involving transformation according to specific rules

### Concrete Examples of Questions
1. "If a train travels at 60 mph for 2 hours, then slows to 40 mph for 1 hour, what is the average speed for the journey?"
   - Auto-CoT would generate reasoning steps for calculating distance and then dividing by total time

2. "In a class of 30 students, 18 study math, 15 study science, and 10 study both. How many students study neither subject?"
   - Auto-CoT would generate set theory reasoning that can serve as a template for similar problems

3. "A shopkeeper buys an item for $80 and marks it up by 25%. During a sale, he offers a 10% discount. What is the final selling price?"
   - Auto-CoT would generate markup and discount calculation steps that can be reused

4. "If 5 workers can build 5 walls in 5 days, how many days would it take 10 workers to build 10 walls?"
   - Auto-CoT would generate proportional reasoning steps that can serve as examples

5. "A code replaces each letter with the letter that is 3 positions later in the alphabet. If 'HELLO' is encoded, what would be the result?"
   - Auto-CoT would generate letter-by-letter transformation steps useful for similar encoding problems

Auto-CoT is particularly effective for these types of questions because:
- They involve clear, traceable reasoning steps that can be automatically generated
- The reasoning patterns are generalizable across similar problems
- The solutions benefit from explicit intermediate calculations
- The generated examples can effectively demonstrate the problem-solving approach
- The step-by-step nature aligns perfectly with how Chain-of-Thought reasoning works

This approach is especially powerful when dealing with problems that follow identifiable patterns of reasoning where seeing similar examples with explicit reasoning steps significantly improves performance on new instances of similar problems.

In [5]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the Auto-CoT prompt and extract the final answer
def generate_auto_cot_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = auto_cot_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final solution from Step 4
    final_answer_pattern = r'Step 4:([\s\S]*?)(?=$|Step \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 4: \[Final Solution\]([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('auto_cot_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL SOLUTION (STEP 4) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_auto_cot", generate_auto_cot_node)

# Graph configuration
graph_builder.set_entry_point("generate_auto_cot")
graph_builder.add_edge("generate_auto_cot", END)
graph = graph_builder.compile()

# Usage example
inputs = {
    "messages": [HumanMessage(content="If a train travels at 60 mph for 2 hours, then slows to 40 mph for 1 hour, what is the average speed for the journey?")],
    "question": None,
    "full_response": None
}

# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL SOLUTION:")

# Graph execution - only display final solution
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_auto_cot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'auto_cot_result.txt')")

QUESTION:
If a train travels at 60 mph for 2 hours, then slows to 40 mph for 1 hour, what is the average speed for the journey?

FINAL SOLUTION:
Final Solution  
The average speed for the journey is approximately 53.33 mph. This is calculated by dividing the total distance of 160 miles by the total time of 3 hours.

(Complete results saved in 'auto_cot_result.txt')
